# Introduction

This module is exposing a part of the WebGL2 context.  Is is assumed that you are familiar with the concepts and commands.
You can find more information about it here : https://webgl2fundamentals.org/

There is some major differences still :

- All the WebGL2 commands are called on the GLViewer instead of a gl context.
- All the API is written in *snake_case* instead of *camelCase*, so for example ``gl.drawArrays(...)`` in JavaScript becomes ``widget.draw_arrays(...)`` in Python
- Masks parameters are replace by positional attribute, so for example ``gl.clear(gl.DEPTH_BUFFER_BIT | gl.COLOR_BUFFER_BIT);`` in JavaScript becomes ``widget.clear(depth_buffer_bit=True, color_buffer_bit=True)`` in Python
- Enums are replaced by strings, so for example ``gl.bufferData(gl.ARRAY_BUFFER, data, gl.DYNAMIC_DRAW);`` in JavaScript becomes ``widget.buffer_data("array_buffer", data, "dynamic_draw")`` in Python
- Some low level functions are replaced by helper methods. For example the compile method on the a GLProgramWidget or the bind method on the GLVerteArrayWidget class.

Not all the functions are exposed as of today.
If you need more, feel free to ask on github https://github.com/JeromeEippers/ipywebgl.

All the commands you call on the GLViewer are push to a commands buffer. That commands buffer is only flushed when you call the render() method.

## Very first triangle

Let's create a simple webgl renderer with a single triangle in it.

### Import
Import the ipywebgl module and numpy

In [1]:
import ipywebgl
import numpy as np

### Viewer
Create an instance of the viewer and change the clear color (to see the canvas) and render that change.


In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.render()
w

GLViewer(camera_pos=[0, 50, 200])

### Program
Create the simple program to display a triangle in clip space

In [3]:
pr = w.create_program()
pr.compile(
"""#version 300 es

in vec4 a_position;
 
void main() {
  gl_Position = a_position;
}
"""
,
"""#version 300 es
precision highp float;

out vec4 outColor;
 
void main() {
  outColor = vec4(1, 0, 0.5, 1);
}
"""
)
pr

GLProgramWidget(uid=0)

### Buffer
Create a buffer to store the vertices values and fill it with 2d positions.

The buffer has 3 vertices with X Y values

In [4]:
buf = w.create_buffer()
buf.update(src_data=np.array(
    [ 0, 0,
      0, 0.5,
      0.7, 0,
    ], dtype=np.float32))
buf

GLBufferWidget(uid=0)

### Vertex Array
Create a vertex array and bind the program and the buffer.

* first argument is the program
* second argument is an array of tuple with
    * the buffer
    * description of the attribute binding
        *supported values are ['1','2','3','4']['i8', 'i16', 'i32', 'u8', 'u16', 'u32', 'f16', 'f32']
    * (n times) name of the n^th attribute in the program

In [5]:
vao = w.create_vertex_array()
vao.bind(
    pr, 
    [
        (buf, "2f32", "a_position"),
    ]
)
vao

GLVertexArrayWidget(uid=0)

### Draw
Update the commands buffer to render that triangle, and call render to send it to the frontend

In [6]:
w.clear()
w.use_program(pr)
w.bind_vertex_array(vao)
w.draw_arrays('triangles', 0, 3)
w.render()

### Move the GLViewer
By redisplaying the viewer here we move it down here.

In [7]:
w

GLViewer(camera_pos=[0, 50, 200])

## Uniform
We will update the shader to use an uniform to move the triangle around.

<i>When displaying the info of the program, it doesn't show the location for the uniform yet. We only look for the location of the uniforms when we try to set them.</i>

In [8]:
pr.compile(
"""#version 300 es
in vec4 a_position;
uniform vec2 u_pos;

void main() {
  gl_Position = a_position + vec4(u_pos,0,0);
}
"""
,
"""#version 300 es
precision highp float;
 
out vec4 outColor;
 
void main() {
  outColor = vec4(1, 0, 0.5, 1);
}
"""
)
pr

GLProgramWidget(uid=0)

### Render
Change the commands buffer to render the program with the uniform now.

In [9]:
w.clear()
w.use_program(pr)
w.set_uniform('u_pos', np.asarray([0.5,0.5], dtype=np.float32))
w.bind_vertex_array(vao)
w.draw_arrays('triangles', 0, 3)
w.render()

Displaying the program now, will show the uniform location info

In [10]:
pr

GLProgramWidget(uid=0)

## Interactive
Let's tweak the triangle position directly in the notebook using the interact function.

In [11]:
from ipywidgets import widgets, interact

def move_triangle(x, y):
    w.clear()
    w.use_program(pr)
    w.set_uniform('u_pos', np.asarray([x, y], dtype=np.float32))
    w.bind_vertex_array(vao)
    w.draw_arrays('triangles', 0, 3)
    w.render()
    
interact(
    move_triangle, 
    x=widgets.FloatSlider(min=-1, max=1, step=.01, value=0),
    y=widgets.FloatSlider(min=-1, max=1, step=.01, value=0)
)
w

interactive(children=(FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.01), FloatSlider(value…

GLViewer(camera_pos=[0, 50, 200])